In [ ]:
import os
import pandas as pd
import cv2

In [ ]:
base_dir = "../input/dog-breed-identification/"

In [ ]:
data = pd.read_csv(base_dir+"labels.csv")

In [ ]:
breed_count = dict(data.breed.value_counts().items())
max_cnt = max(breed_count.values())

In [ ]:
import numpy as np
bal_data = list(data.values)
for breed in breed_count:
  diff_data = data[data.breed==breed].sample(max_cnt-breed_count[breed],replace=False)
  bal_data.extend(list(diff_data.values))
new_data = pd.DataFrame(bal_data,columns=data.columns)

In [ ]:
new_data['id'] = new_data['id'].apply(lambda x:x+'.jpg')

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(new_data,random_state=42,test_size=500,stratify=new_data.breed)
x_train.shape, x_test.shape

In [ ]:
x_train.breed.value_counts(),x_test.breed.value_counts()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Flatten,Activation,Dropout,BatchNormalization,Input,Softmax, concatenate
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:
from sklearn.metrics import f1_score
def f1_score_micro1(y_true, y_pred):
  y_prediton = (tf.math.argmax(y_pred, axis = -1))
  y_actual = (tf.math.argmax(y_true, axis = -1))
  f1 = f1_score(y_actual,y_prediton,average='micro', zero_division = 0)
  return f1
def f1_score_micro(y_true, y_pred):
  return tf.py_function(f1_score_micro1, (y_true, y_pred), tf.float32)

In [ ]:
from sklearn.metrics import log_loss
def log_loss1(y_true, y_pred):
  f1 = log_loss(y_true, y_pred)
  return f1
def logloss(y_true, y_pred):
  return tf.py_function(log_loss1, (y_true, y_pred), tf.float32)

In [ ]:
log_dir='./logs/fit'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)

In [ ]:
nasnet = tf.keras.applications.NASNetLarge(
    input_shape=(331, 331, 3),
    include_top=False,
    weights="imagenet",
)

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.1)

train_generator=datagen.flow_from_dataframe(
dataframe=x_train,
directory=base_dir+"train/",
x_col="id",
y_col="breed",
subset="training",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(331, 331),
color_mode='rgb')

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=x_train,
directory=base_dir+"train/",
x_col="id",
y_col="breed",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(331, 331),
color_mode='rgb',)

In [ ]:
nasnet.trainable = False

In [ ]:
class learng_rate_schedule(tf.keras.callbacks.Callback):
    def __init__(self,verbose=1):
        super(learng_rate_schedule, self).__init__()
        self.verbose=verbose
    def on_epoch_begin(self,epoch,verbose=1):
        history=self.model.history.history
        lr=float(K.get_value(self.model.optimizer.lr))
        if epoch%3==2:
            lr=lr*0.95
        elif (len(history.get('val_logloss',[]))>=2 and epoch%3!=2):
            if history['val_logloss'][-1]<=history['val_logloss'][-2]:
                lr=lr*0.9
            else:
                lr=lr
        else:
            lr=lr
        K.set_value(self.model.optimizer.lr, K.get_value(lr))
        if self.verbose > 0:
            print('\nEpoch %05d: LearningRateScheduler reducing learning rate to %s.' % (epoch + 1, lr))
        
    def on_epoch_end(self, epoch, logs):
        logs = logs
        logs['lr'] = K.get_value(self.model.optimizer.lr)


filepath=log_dir+"/save/"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_logloss',  verbose=1, save_best_only=True, mode='min')


In [ ]:
nasnet.output

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()
layer3=Conv2D(128,(7,7),activation='relu',padding="valid")(nasnet.output)
drop = Dropout(0.2)(layer3)
layer4=Conv2D(128,(3,3),activation='relu',padding="valid")(drop)
drop1 = Dropout(0.2)(layer4)
layer5=Conv2D(128,(3,3),activation='relu',padding="valid")(drop1)
layer6=Flatten()(layer5)
drop2 = Dropout(0.2)(layer6)
output_layer=Dense(120,activation='softmax')(drop2)
model3=Model(inputs=nasnet.input,outputs=output_layer)
model3.compile(optimizer='adam',
 loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[f1_score_micro,logloss]
 )

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
model3.fit_generator(generator=train_generator,
                      steps_per_epoch=STEP_SIZE_TRAIN,
                      validation_data=valid_generator,
                      validation_steps=STEP_SIZE_VALID,
                      epochs=10, callbacks = [checkpoint,tensorboard_callback])

In [ ]:
import shutil
shutil.make_archive('./logs', 'zip', "./")